## fitpet 콘텐츠 크롤링 (10번 데이터)

- 각 콘텐츠 주소(건강/생활/음식/제품) -> 페이지 별 링크 수집 -> 링크 접속 후 제목, 이미지, 본문 추출 -> csv 저장

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# 크롤링할 주소 및 페이지 수 정의
category_urls = [
    'https://www.fitpetmall.com/blog/category/dogs/dog-health',
    'https://www.fitpetmall.com/blog/category/dogs/dog-living',
    'https://www.fitpetmall.com/blog/category/dogs/dog-nutrition',
    'https://www.fitpetmall.com/blog/category/dogs/dog-products'
]

max_pages_list = [6, 7, 4, 5]


# 주소 url 수집 함수 정의
def get_article_links(start_url, max_pages):
    all_links = []
    current_page = 1
    url = start_url

    while current_page <= max_pages:
        print(f"🔍 스크래핑 중: {start_url} 페이지 {current_page}")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")

        # 본문 URL 추출
        article_tags = soup.select("div.post-content-wrap a.entire-meta-link")
        page_links = [tag['href'] for tag in article_tags if tag.has_attr('href')]
        all_links.extend(page_links)

        # 다음 페이지 URL 찾기
        next_page_tag = soup.select_one("a.next.page-numbers")
        if next_page_tag:
            url = next_page_tag['href']
            current_page += 1
            time.sleep(1)  # 서버 과부하 방지
        else:
            print("⛔ 다음 페이지 없음")
            break

    return all_links



In [ ]:
# 전체 URL 수집
all_article_urls = []

for idx, category_url in enumerate(category_urls):
    max_pages = max_pages_list[idx]
    links = get_article_links(category_url, max_pages=max_pages)
    all_article_urls.extend(links)

print(f"\n총 {len(all_article_urls)}개의 글 URL을 수집했습니다.")

# 결과 출력
for link in all_article_urls:
    print(link)

In [ ]:
# CSV 파일 저장 (제목, 본문, URL, 대표 이미지)

with open('FINAL_TOTAL_FITPET_DATA.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['title', 'content', 'URL', 'image'])

    for url in all_article_urls:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"⚠️ Failed to fetch {url}")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        # 제목 가져오기
        title_tag = soup.select_one('h1.entry-title')
        title = title_tag.get_text(strip=True) if title_tag else '제목 없음'

        # 본문 가져오기
        content_container = soup.select_one('div.gusta-post-content')
        if content_container:
            content_parts = []
            for tag in content_container.find_all(['p', 'figcaption']):
                text = tag.get_text(strip=True)
                if text:
                    content_parts.append(text)
            content = '\n'.join(content_parts) if content_parts else '내용 없음'
        else:
            content = '내용 없음'

        # 대표 이미지 가져오기
        og_image_tag = soup.select_one('meta[property="og:image"]')
        og_image = og_image_tag['content'] if og_image_tag and og_image_tag.has_attr('content') else '이미지 없음'

        # CSV에 쓰기
        writer.writerow([title, content, url, og_image])

print("✅ CSV 저장 완료!")


In [ ]:
# 필요 시
import pandas as pd

# CSV 파일 읽어오기
df = pd.read_csv('FINAL_TOTAL_FITPET_DATA.csv', encoding='utf-8-sig')

# 컬럼명 변경
df.columns = ['title', 'content', 'URL', 'image']
print(df.head())

# 변경된 데이터프레임을 다시 저장
df.to_csv('FINAL_TOTAL_FITPET_DATA.csv', index=False, encoding='utf-8-sig')

print("✅ 컬럼명 변경 및 CSV 저장 완료!")

                                    title  \
0                     강아지 생리 | 주기부터 대처법까지   
1                           강아지 항문낭 짜는 방법   
2      수의사가 말하는 반려동물 구강관리가 아이의 행복에 미치는 영향   
3     수의사 칼럼 ‘엉덩이를 실룩거리는 강아지 걸음걸이는 위험하다?’   
4  수의사 칼럼 ‘”산책하다 주저앉았다” 강아지 디스크 어떻게 치료하나’   

                                             content  \
0  강아지와 함께 생활하면서 강아지가 겪는 변화들이 때로는 반려인을 놀라게 하죠. 하지...   
1  강아지 항문낭은 강아지의 항문 주위에 위치한 작은 주머니로, 배변 활동 시 항문낭액...   
2  안녕하세요. 건강한 반려생활의 시작, 핏펫입니다.오늘은 넬 동물의료센터 원장 엄태흠...   
3  ❚윤용석 샤인동물메디컬센터 대표원장\n몇 달 전 병원에 1살 된 예쁜 푸들, 크림(...   
4  ❚신영식 샤인동물메디컬센터 외과 원장\n5살 토리(페키니즈·수컷)는 산책하다가 갑자...   

                                                 URL  \
0  https://www.fitpetmall.com/blog/dogs-menstruat...   
1   https://www.fitpetmall.com/blog/dog-anal-glands/   
2  https://www.fitpetmall.com/blog/oral-health-an...   
3  https://www.fitpetmall.com/blog/%ec%88%98%ec%9...   
4  https://www.fitpetmall.com/blog/%ec%88%98%ec%9...   

                                               image  
